In [1]:
import os, cv2, dlib, re, pyrender, trimesh
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation 

In [2]:
def loadObj(file):
    obj = {}
    obj['v'],obj['vn'],obj['vt'],obj['f'] = [],[],[],[]
    f = open(file, "r")
    lines = f.readlines()
    delimiters = " ", "//", "\\","/","\\"
    regexPattern = '|'.join(map(re.escape, delimiters))
    for item in lines:
        data = re.split(regexPattern, item)
        if 'v' == data[0]:
            obj['v'].append(np.array(data[1:]).astype('float'))
        elif 'vt' == data[0]:
            obj['vt'].append(np.array(data[1:]).astype('float'))
        elif 'vn' == data[0]:
            obj['vn'].append(np.array(data[1:]).astype('float'))
        elif 'f' == data[0]:
            obj['f'].append(np.array(data[1:]).astype('int32'))
    
    hasVN,hasVT = False,False
    if len(obj['v']):
        obj['v'] = np.vstack(obj['v'])
    if len(obj['vn']):
        hasVN = True
        obj['vn'] = np.vstack(obj['vn'])
    if len(obj['vt']):
        hasVT = True
        obj['vt'] = np.vstack(obj['vt'])
    if len(obj['f']):
        obj['f'] = np.vstack(obj['f'])
    
    f = np.ones((9,obj['f'].shape[0]))
    if  hasVN and hasVT:
        f = obj['f'][:,[0,3,6,2,5,8,1,4,7]].T
    elif hasVN and obj['f'].shape[1]==6:
        f[:6] = obj['f'][:,[0,2,4,1,3,5]].T
    elif hasVT and obj['f'].shape[1]==6:
        f[[0,1,2,6,7,8]] = obj['f'][:,[0,2,4,1,3,5]].T
    else:
        f[[0,1,2]] = obj['f'].T
    obj['f'] = f.astype('uint32')-1
    return obj


def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

def landmark_detect(gray):
    rects = detector(gray, 1)
    if len(rects)==0:
        print('Couldn\'t detect faces on the image.')
        return
        
    rect = rects[0]
    shape = predictor(gray, rect)
    lm = np.array([[p.x, p.y] for p in shape.parts()])
    return lm

def angle2matrix(pose):
    ''' compute Rotation Matrix from three Euler angles
    '''
    R_x = np.array([[1, 0, 0],
                    [0, np.cos(pose[0]), -np.sin(pose[0])],
                    [0, np.sin(pose[0]), np.cos(pose[0])]
                    ])

    R_y = np.array([[np.cos(pose[1]), 0, np.sin(pose[1])],
                    [0, 1, 0],
                    [-np.sin(pose[1]), 0, np.cos(pose[1])]
                    ])

    R_z = np.array([[np.cos(pose[2]), -np.sin(pose[2]), 0],
                    [np.sin(pose[2]), np.cos(pose[2]), 0],
                    [0, 0, 1]
                    ])
    R = np.dot(R_z, np.dot(R_y, R_x))
    return R


def isRotationMatrix(R):
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype=R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6


def matrix2angle(R):
    #assert (isRotationMatrix(R))

    sy = np.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])

    singular = sy < 1e-6

    if not singular:
        x = np.arctan2(R[2, 1], R[2, 2])
        y = np.arctan2(-R[2, 0], sy)
        z = np.arctan2(R[1, 0], R[0, 0])
    else:
        x = np.arctan2(-R[1, 2], R[1, 1])
        y = np.arctan2(-R[2, 0], sy)
        z = 0

    return np.array([x, y, z])

def render(mesh,cam,model=np.eye(4),f=[750,750],resolution=[512,512]):
    # f [fx,fy]
    fx, fy = f[0], f[1]
    obj = pyrender.Mesh.from_trimesh(mesh, smooth=False)
    fovY = 2*np.arctan(resolution[0]/2/fy)
    # compose scene
    scene = pyrender.Scene(ambient_light=[1.0, 1.0, 1.0], bg_color=[0, 0, 0])
    camera = pyrender.PerspectiveCamera( yfov=fovY)
    light = pyrender.SpotLight(color=[1.0, 1.0, 1.0], intensity=3,innerConeAngle=0.05, outerConeAngle=0.5)
#     light = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=1.0)

    scene.add(obj, pose=  model)
    scene.add(light, pose=  cam)
    scene.add(camera, pose = cam)
    
    # render scene
    r = pyrender.OffscreenRenderer(resolution[1], resolution[0])
    color, depth = r.render(scene)
    return color,depth



def save_to_pts(lm,file):
    f = open(file,'w')
    print('version: 1\nn_points:  68\n{',file=f)
    for i in range(68):
        print('%d %d'%(lm[i,0],lm[i,1]),file=f)
    print('}',file=f)
    f.close()
    
def shape_to_np(shape, dtype="int"):
    coords = np.zeros((68, 2), dtype=dtype)

    for i in range(0, 68):
        coords[i] = (np.round(shape.part(i).x), np.round(shape.part(i).y))

    return coords

def ortho(frustum):
    left,right,bottom,top = frustum[:]
    projection = np.eye(4)
    projection[0,0], projection[1,1],projection[2,2] = 2/(right - left), 2/(top - bottom), -1
    projection[0,3],projection[1,3] = -(right + left) / (right - left),  -(top + bottom) / (top - bottom)
    return projection



In [3]:
shape_predictor = '../shape_predictor_68_face_landmarks_dlib.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor)

In [6]:
def render2(mesh,cam=np.eye(4),model=np.eye(4),scale=[1,1],resolution=[720,1280]):

    obj = pyrender.Mesh.from_trimesh(mesh, smooth=False)

    scene = pyrender.Scene(ambient_light=[1.0, 1.0, 1.0], bg_color=[0, 0, 0])
    camera = pyrender.OrthographicCamera(xmag=1, ymag=resolution[0]/2/scale[1], znear=1e-3, zfar=10000.0)
    light = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=2.0)

    scene.add(obj, pose=  model)
    scene.add(light, pose=  model)
    scene.add(camera, pose = cam)
    
    # render scene
    r = pyrender.OffscreenRenderer(resolution[1], resolution[0])
    color, _ = r.render(scene)
    
    width = int(round(resolution[1]*scale[0]/scale[1]))
    img = cv2.resize(color,(width,resolution[0]))
    pad = (width - resolution[1])//2
    right_pad = resolution[1] - width - abs(pad)
    if pad > 0:
        img = img[:,pad:pad+resolution[1]]
    elif pad < 0:
        img = np.pad(img, ((0,0),(abs(pad),right_pad),(0,0)), 'edge')
    
    return img,_

def solveCamera(model_points,image_points):
    # model_points, image_points N*M
    A,b = np.zeros((2*model_points.shape[0],8)),np.zeros((2*model_points.shape[0],1))
    ind = 0
    for model_point, image_point in zip(model_points,image_points):
        A[2*ind,:3],A[2*ind+1,4:7] = model_point[:3],model_point[:3]
        A[2*ind,3],A[2*ind+1,7] = 1,1
        b[2*ind:2*ind+2] = image_point.reshape((-1,1))
        ind += 1
    affine,_,_,_ = np.linalg.lstsq(A,b,rcond=None)
    affine = affine.reshape((2,4))
    
    norms = np.linalg.norm(affine[:,:3],axis=1)
    scale = norms
    R = affine[:,:3]/norms.reshape((-1,1))
    R1,R2 = R[0],R[1]
    R3 = np.cross(R1,R2)
    R3 /= np.linalg.norm(R3)
    R = np.vstack((R1,R2,R3))

    U,S,V = np.linalg.svd(R)
    R_ortho = np.dot(U, V)
    if np.linalg.det(R_ortho) < 0:
        U[2,0] = -U[2,0]
        R_ortho = np.dot(U, V)
    t1,t2 = affine[0,3],affine[1,3]

    MV = np.zeros((4,4))
    MV[:3,:3] = R_ortho[:3]
    MV[:2,3]  = (affine[:2,3] - np.array(img_size)/2)/scale
    MV[3,3],MV[2,3] = 1, -200
    
    t1,t2 = affine[0,3]/scale[0],affine[1,3]/scale[1]
    view_model = np.eye(4)
    view_model[:3,:3] = R_ortho[:3]
    view_model[:2,3]  = np.array([t1,t2])
    frustum = np.array([0.0,img_size[0]/scale[0],0.0,img_size[1]/scale[1]])
    ortho_projection = ortho(frustum)
    mvp = np.dot(ortho_projection, view_model)
    mvp[2,3] = -2
    viewport = np.array([0, img_size[1], img_size[0], -img_size[1]]) # flips y, origin top-left, like in OpenCV
    viewport_mat = np.array( [[viewport[2] / 2.0, 0.0, 0.0, viewport[2] / 2.0 + viewport[0]], \
                              [0.0,               viewport[3] / 2.0, 0.0, viewport[3] / 2.0 + viewport[1]], \
                              [0.0,               0.0,               1.0, 0.0], \
                              [0.0,               0.0,               0.0, 1.0]]) 
    full_projection = np.dot(viewport_mat, mvp)

    return MV, affine, scale, full_projection

def findNearestCooresponding(image_points,model_points,affine):
    points = np.hstack((model_points,np.ones((model_points.shape[0],1))))
    reProjection = np.dot(affine,points.T).T
    model_ids = []
    for item in image_points:
        dis = np.sum(np.abs(reProjection - item.reshape((1,-1))),axis=1)
        model_ids.append(np.argmin(dis))
    return model_points[model_ids],model_ids

def visLM(model_points, image_points, im,affine, homogeneous=False):
    if homogeneous:
        points = np.hstack((model_points,np.ones((model_points.shape[0],1))))
    else:
        points = model_points
    points = np.dot(affine,points.T).T
    for p in points:
        cv2.circle(im, (int(p[0]), int(im.shape[0] - p[1])), 3, (0,255,0), -1)  
    for p in image_points:
        cv2.circle(im, (int(p[0]), int(im.shape[0] - p[1])), 3, (0,0,255), -1)  
    return im

def calReprojecErr(affine,point,lm):
    point_img = np.dot(affine[:2,:3],point.T)+affine[:2,[3]]
    return np.mean(np.abs(lm-point_img.T))                 
    
def buildEdgeTopology(mesh):
    edges = []
    for item in mesh.faces:
        edges.append([item[0],item[1]])
        edges.append([item[1],item[2]])
        edges.append([item[2],item[0]])
    return np.vstack(edges)

def isFront(mesh,point_id):
    z = mesh.vertices[:,2][point_id]
    mask = z[:,0]>z[:,1]
    front_id = np.unique(np.hstack((point_id[mask,0],point_id[~mask,1])))
    return front_id

def pointCloudClustering(vertices,thread):
    isTravel = np.zeros(vertices.shape[0]).astype('bool')
    while np.sum(isTravel) < vertices.shape[0]:
        sample = np.where(isTravel)[0]
        point = vertices[sample]
        dis = np.sum(np.abs(vertices - point.reshape((1,-1))),axis=0)/3
        ind = np.where(dis<thread)
#         if isTravel[ind]
    
def findOcclusionEdge(mesh,MV,affine, contourLM, edge):
    normal = np.dot(MV[:3,:3],mesh.vertex_normals.T).T
    isBoundaries = normal[:,2][edge]>0
    isBoundaries = isBoundaries[:,0]!=isBoundaries[:,1]
    isBoundaries = np.where(isBoundaries)
    point_id = []
    for item in isBoundaries:
        point_id.append(edge[item])
    point_id = np.hstack(point_id)
    
    # Compute vertices that lye on occluding boundaries:
    point_id = isFront(mesh,point_id)
    points = mesh.vertices[point_id]
    
    # Project 3D boundaries to 2D
    points = np.hstack((points,np.ones((points.shape[0],1))))
    points_2D = np.dot(affine,points.T).T
    
    # Filter points far away
    ids,ids_img = [],[]
    threadHood = np.sum(np.abs(contourLM[0]-contourLM[-1]))/25
    for i,item in enumerate(points_2D):
        if  not contourMask[point_id[i]]:
            continue
        dis = np.sum(np.abs(contourLM-item.reshape((1,-1))),axis=1)
        minimal_id = np.argmin(dis)
        if dis[minimal_id] < threadHood:
            ids.append(i)
            ids_img.append(minimal_id)
    point_id = point_id[ids]
    return mesh.vertices[point_id],contourLM[ids_img], point_id

def fitOccludingContour(angle,MV,affine,lm,image_ids,edge):
    if angle[1]>= 0:
        contour = lm[image_ids[1]]
    else:
        contour = lm[image_ids[0]]
        
    return findOcclusionEdge(mesh,MV,affine,contour,edge)

def optimizeExp(obj_base,affine,w_exp,lm,ids,num_coeffs_to_fit=10,lamb=10.0):
    #  affine*(v_base+ w_base*coef) = lm
    # v_base n*3, w_base n*useBasis, coef useBasis*1, affine 3*4

    lm_base, v_base = lm[ids['image_exp_ids']], obj_base.reshape((-1,3))[ids['model_exp_ids']]#
    w_base = w_exp.reshape((-1,3,w_exp.shape[-1]))[ids['model_exp_ids']][...,:num_coeffs_to_fit].reshape((-1,num_coeffs_to_fit))
    
#     print(calReprojecErr(affine,v_base,lm_base))
    
    num_landmarks = v_base.shape[0]
    y,v_bar = np.ones(3*num_landmarks),np.ones(4*num_landmarks)
    V_hat_h = np.zeros((4 * num_landmarks, num_coeffs_to_fit))
    P = np.zeros((3 * num_landmarks, 4 * num_landmarks))
    for i in range(num_landmarks):
        V_hat_h[i*4:i*4+3] = w_base[i*3:i*3+3]
        P[i*3:i*3+3,i*4:i*4+4] = affine
        y[i*3:i*3+2] = lm_base[i]
        v_bar[4*i:4*i+3] = v_base[i]
        
    sigma_squared_2D = 3
    Omega = np.eye(3 * num_landmarks)/sigma_squared_2D
    
    A = np.dot(P, V_hat_h)
    b = np.dot(P, v_bar) - y
    AtOmegaAReg = np.dot(np.dot(A.T,Omega), A) + lamb * np.eye(num_coeffs_to_fit)
    rhs = - np.dot(np.dot(A.T, Omega), b)
    
    
    # solve with qr
    Q, R = np.linalg.qr(AtOmegaAReg)
    y = np.dot(Q.T, rhs)
    param = np.linalg.solve(R, y).reshape((-1,1))
    obj_with_exp = obj_base + np.dot(w_exp[...,:num_coeffs_to_fit],param)
#     print(calReprojecErr(affine,obj_with_exp.reshape((-1,3))[ids['model_exp_ids']],lm_base))
    return obj_with_exp,param

def optimizePose(mesh,lm,ids,steps=10): 
    
    image_5point_ids,model_5point_ids,model_ids,image_ids = ids['image_5point_ids'],ids['model_5point_ids'],ids['model_ids'],ids['image_ids']
    model_points = mesh.vertices[model_5point_ids].astype('float32')
    image_points = lm[image_5point_ids].astype('float32')
    MV,affine,scale,full_projection = solveCamera(model_points,image_points)

    for iter in range(steps):

        threadHood = 7.5/180*np.pi
        angle = matrix2angle(MV[:3,:3])

        model_points = mesh.vertices[model_5point_ids].astype('float32')
        image_points = lm[image_5point_ids].astype('float32')
        model_ids_new = np.array(model_5point_ids)

        if  angle[1] < threadHood:
            image_points_add, model_points_add = lm[image_ids[1]],mesh.vertices[model_ids[1]]
            model_points_add, model_ids_add = findNearestCooresponding(image_points_add, model_points_add,affine)
            
            image_points = np.vstack((image_points,image_points_add))
            model_points = np.vstack((model_points,model_points_add))
            model_ids_new = np.hstack((model_ids_new,model_ids[1][model_ids_add]))

        if angle[1] > -threadHood:
            image_points_add, model_points_add = lm[image_ids[0]],mesh.vertices[model_ids[0]]
            model_points_add, model_ids_add = findNearestCooresponding(image_points_add, model_points_add,affine)

            image_points = np.vstack((image_points,image_points_add))
            model_points = np.vstack((model_points,model_points_add))
            model_ids_new = np.hstack((model_ids_new,model_ids[0][model_ids_add]))

#         MV,affine,scale,full_projection = solveCamera(model_points,image_points)
        contour_3D,contour_2D,model_ids_add = fitOccludingContour(angle,MV,affine,lm,image_ids,edge)
        image_points = np.vstack((image_points,contour_2D))
        model_points = np.vstack((model_points,contour_3D))
        model_ids_new = np.hstack((model_ids_new,model_ids_add))
       
        MV,affine,scale,full_projection = solveCamera(model_points,image_points)
        affine2 = np.vstack((affine,np.array([0,0,0,1])))
        vertice,param = optimizeExp(obj_base,affine2,models['w_exp'],lm,ids)
    
    mesh.vertices = vertice.reshape((-1,3))
    
    return MV,affine,scale,image_points,model_points,full_projection,model_ids_new

In [17]:
imageFolder = 'video.mp4'
vc = cv2.VideoCapture(imageFolder)
frameCount,maxFrame,sampling = 0,100,1
success, im = vc.read()
mesh = trimesh.load('obama.obj',process=False)
contourMask = np.load('contourMask.npy')
edge = buildEdgeTopology(mesh)

image_mouse_ids = np.array(range(49,69))-1
model_mouse_ids = np.array([5522,6026,7355,8181,9007,10329,10857,9730,8670,8199,7726,6898,6291,7364,8190,9016,10088,8663,8191,7719])
image_5point_ids = np.hstack((np.array([30,8]),np.array(range(36,48))))
model_5point_ids = np.hstack((np.array([8156,47846]),np.array([2602,4147,4921,6088,4931,4157,10390,11031,11932,13481,12072,11298])))
image_5point_ids,model_5point_ids = np.hstack((image_5point_ids,image_mouse_ids)),np.hstack((model_5point_ids,model_mouse_ids))


model_ids = np.array([[22057,22209,21582,21350,42838,44845,46470,47262], \
                      [48432, 49307,50856,52755,33087,32933,32141,32249]])# right left

image_ids = np.array([[0,1,2,3,4,5,6,7],[9,10,11,12,13,14,15,16]])
image_exp_ids = image_mouse_ids #np.array(range(61,69))-1
model_exp_ids = model_mouse_ids #np.array([6291,7364,8190,9016,10088,8663,8191,7719 ])
# image_exp_ids = np.hstack((np.array(range(37,49)),np.array(range(61,69))))-1
# model_exp_ids = np.array([2602,4147,4921,6088,4931,4157,10390,11031,11932,13481,12072,11298,6291,7364,8190,9016,10088,8663,8191,7719 ])
ids = {'image_5point_ids':image_5point_ids,'model_5point_ids':model_5point_ids,'model_ids':model_ids,'image_ids':image_ids, \
       'image_exp_ids':image_exp_ids, 'model_exp_ids':model_exp_ids}
  
# face model
filename = './Model_Shape.mat'
models = loadmat(filename)

sigma_exp,sigma = models['sigma_exp'],models['sigma']
models['w'] = models['w']*np.repeat(sigma,models['w'].shape[0],axis=1).T
models['w_exp'] = models['w_exp']*np.repeat(sigma_exp,models['w_exp'].shape[0],axis=1).T
shape_coef = np.loadtxt('./00000.jpg.coeffs_pca_shape.txt').reshape((-1,1))
#exp_coef = np.loadtxt('E:/mesh2face/pose_estimation/eos-expression-aware-proxy/build/examples/data/res/00000.jpg.coeffs_expression.txt').reshape((-1,1))
obj_base = models['mu_shape'] + models['mu_exp'] + np.dot(models['w'],shape_coef) #+ np.dot(models['w_exp'],exp_coef)
mesh.vertices = obj_base.reshape((-1,3))

lms = np.load('obama_lm.npy')
while success and frameCount < maxFrame:
    
    img_size = (im.shape[1], im.shape[0])
    lm = lms[frameCount,0]
    lm[:,1] = img_size[1] - lm[:,1]
    
    mesh.vertices = obj_base.reshape((-1,3))
    MV,affine,scale,image_points,model_points,full_projection,model_ids = optimizePose(mesh,lm,ids,steps=3)

    
    renderedImg, depth = render2(mesh,model=MV,scale=scale)
    mask = np.sum(renderedImg,axis=2)>0
    im[mask] = renderedImg[mask]*0.5 + im[mask]*0.5

    im = visLM(mesh.vertices[model_ids],image_points,im, affine,homogeneous=True)
#     im = visLM(mesh.vertices[model_exp_ids],lm[image_exp_ids],im, affine,homogeneous=True)
    
#     plt.imshow(img[...,::-1])
#     p1 = ( int(image_points[-3][0]), int(image_points[-3][1]))
#     p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))
#     cv2.line(im, p1, p2, (255,0,0), 2)
    
    
    shape, scale = im.shape, 1
    cv2.imshow("Output", cv2.resize(im,(shape[1]//scale,shape[0]//scale)))
    if  cv2.waitKey(33) & 0xFF == ord('q'):#
        cv2.destroyAllWindows()
        break
    
#     np.savetxt('optimize_v1/%03d.txt'%frameCount,model_points)
    cv2.imwrite('optimize_v3/%03d.jpg'%frameCount,im)
    frameCount += 1
    if maxFrame -1 >0:
        vc.set(cv2.CAP_PROP_POS_FRAMES, frameCount*sampling)
    success, im = vc.read()
cv2.destroyAllWindows()

unable to load materials from: image_0010.out.mtl
specified material (FaceTexture)  not loaded!


In [10]:
imageFolder = 'video.mp4'
vc = cv2.VideoCapture(imageFolder)
frameCount,maxFrame,sampling = 0,100,1
success, im = vc.read()
mesh = trimesh.load('obama.obj',process=False)
contourMask = np.load('contourMask.npy')
edge = buildEdgeTopology(mesh)

image_mouse_ids = np.array(range(49,69))-1
model_mouse_ids = np.array([5522,6026,7355,8181,9007,10329,10857,9730,8670,8199,7726,6898,6291,7364,8190,9016,10088,8663,8191,7719])
image_5point_ids = np.hstack((np.array([30,8]),np.array(range(36,48))))
model_5point_ids = np.hstack((np.array([8156,47846]),np.array([2602,4147,4921,6088,4931,4157,10390,11031,11932,13481,12072,11298])))
image_5point_ids,model_5point_ids = np.hstack((image_5point_ids,image_mouse_ids)),np.hstack((model_5point_ids,model_mouse_ids))


model_ids = np.array([[22057,22209,21582,21350,42838,44845,46470,47262], \
                      [48432, 49307,50856,52755,33087,32933,32141,32249]])# right left

image_ids = np.array([[0,1,2,3,4,5,6,7],[9,10,11,12,13,14,15,16]])
image_exp_ids = image_mouse_ids #np.array(range(61,69))-1
model_exp_ids = model_mouse_ids #np.array([6291,7364,8190,9016,10088,8663,8191,7719 ])
# image_exp_ids = np.hstack((np.array(range(37,49)),np.array(range(61,69))))-1
# model_exp_ids = np.array([2602,4147,4921,6088,4931,4157,10390,11031,11932,13481,12072,11298,6291,7364,8190,9016,10088,8663,8191,7719 ])
ids = {'image_5point_ids':image_5point_ids,'model_5point_ids':model_5point_ids,'model_ids':model_ids,'image_ids':image_ids, \
       'image_exp_ids':image_exp_ids, 'model_exp_ids':model_exp_ids}
  
# face model
filename = './Model_Shape.mat'
models = loadmat(filename)

sigma_exp,sigma = models['sigma_exp'],models['sigma']
models['w'] = models['w']*np.repeat(sigma,models['w'].shape[0],axis=1).T
models['w_exp'] = models['w_exp']*np.repeat(sigma_exp,models['w_exp'].shape[0],axis=1).T
shape_coef = np.loadtxt('E:/mesh2face/pose_estimation/eos-expression-aware-proxy/build/examples/data/res/00000.jpg.coeffs_pca_shape.txt').reshape((-1,1))
#exp_coef = np.loadtxt('E:/mesh2face/pose_estimation/eos-expression-aware-proxy/build/examples/data/res/00000.jpg.coeffs_expression.txt').reshape((-1,1))
obj_base = models['mu_shape'] + models['mu_exp'] + np.dot(models['w'],shape_coef) #+ np.dot(models['w_exp'],exp_coef)
mesh.vertices = obj_base.reshape((-1,3))

while success and frameCount < maxFrame:
    
    img_size = (im.shape[1], im.shape[0])
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    lm = landmark_detect(gray)
    lm[:,1] = img_size[1] - lm[:,1]
    
    mesh.vertices = obj_base.reshape((-1,3))
    MV,affine,scale,image_points,model_points,full_projection,model_ids = optimizePose(mesh,lm,ids,steps=3)

    
    renderedImg, depth = render2(mesh,model=MV,scale=scale)
    mask = np.sum(renderedImg,axis=2)>0
    im[mask] = renderedImg[mask]*0.5 + im[mask]*0.5

#     im = visLM(mesh.vertices[model_ids],image_points,im, affine,homogeneous=True)
#     im = visLM(mesh.vertices[model_exp_ids],lm[image_exp_ids],im, affine,homogeneous=True)
    
#     plt.imshow(img[...,::-1])
#     p1 = ( int(image_points[-3][0]), int(image_points[-3][1]))
#     p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))
#     cv2.line(im, p1, p2, (255,0,0), 2)
    
    
    shape, scale = im.shape, 1
    cv2.imshow("Output", cv2.resize(im,(shape[1]//scale,shape[0]//scale)))
    if  cv2.waitKey(33) & 0xFF == ord('q'):#
        cv2.destroyAllWindows()
        break
    
#     np.savetxt('optimize_v1/%03d.txt'%frameCount,model_points)
    cv2.imwrite('optimize_v2/%03d.jpg'%frameCount,im)
    frameCount += 1
    if maxFrame -1 >0:
        vc.set(cv2.CAP_PROP_POS_FRAMES, frameCount*sampling)
    success, im = vc.read()
cv2.destroyAllWindows()

# mesh.export('test.obj');
# mesh.vertices = obj_base.reshape((-1,3))
# mesh.export('test2.obj');

unable to load materials from: image_0010.out.mtl
specified material (FaceTexture)  not loaded!


In [4]:
barycentricsModel = np.load('barycentricsModels.npy',allow_pickle=True)[()]
barycentrics, indTri = barycentricsModel['barycentrics'],barycentricsModel['indTri']
barycentrics = np.repeat(barycentrics[...,np.newaxis],3,-1)

for ind in range(384000, 405000,1000):
    im = cv2.imread('./test/%07d_input.png'%ind)
    img_size = (im.shape[1], im.shape[0])

    # image feature points
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    lm = landmark_detect(gray)
    image_points = lm[[36,39,42,45,27,28,29,30,8]].astype('float32')

    # 3d feature points
    mesh = trimesh.load('./test/demo_%07d.obj'%ind,process=False)
    vertices = mesh.vertices[mesh.faces[indTri.astype('int32')]]
    model_points = np.sum(vertices * barycentrics,axis=1).astype('float32')


    focal_length = img_size[0]*4
    center = np.array(img_size)/2
    camera_matrix = np.array([[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )


    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    # retval, camera_matrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(model_points[np.newaxis], image_points[np.newaxis], img_size,camera_matrix,None,flags=cv2.CALIB_USE_INTRINSIC_GUESS)
    # rvec, tvec = rvecs[0],tvecs[0]
    _, rvec, tvec, _ = cv2.solvePnPRansac(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_UPNP )
    r = Rotation.from_rotvec(rvec.reshape((-1))).as_dcm()
    extrins = np.hstack([r,tvec])
    extrins = np.vstack([extrins,np.array([0,0,0,1])])

    # print("Camera Matrix :\n {0}".format(camera_matrix))
    # print("Rotation Vector:\n {0}".format(rotation_vector))
    # print("Translation Vector:\n {0}".format(translation_vector))

    nose_end_point3D = model_points[-2] + np.array([(0.0, 0.0, 0.05)])
    (nose_end_point2D, jacobian) = cv2.projectPoints(nose_end_point3D, rvec, tvec, camera_matrix, dist_coeffs)

    # Display image
    # cv2.imshow("Output", im)
    # cv2.waitKey(0)

    MV = np.eye(4)
    mv = extrins.copy()
    mv[[0,2]] = -mv[[0,2]]
    renderedImg, depth = render(mesh,MV,model=mv,f=[camera_matrix[0,0],camera_matrix[1,1]],resolution=img_size)
    mask,renderedImg = depth[::-1,::-1]>0,renderedImg[::-1,::-1,::-1]
    im[mask] = renderedImg[mask]*0.8 + im[mask]*0.2

    point = np.dot(camera_matrix,np.dot(extrins[:3,:3],model_points.T)+extrins[:3,3].reshape((3,1)))
    point = point[:2]/point[2]
    for p in point.T:
        cv2.circle(im, (int(p[0]), int(p[1])), 2, (0,255,0), -1)  
    for p in image_points:
        cv2.circle(im, (int(p[0]), int(p[1])), 2, (0,0,255), -1)  
    p1 = ( int(image_points[-2][0]), int(image_points[-2][1]))
    p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))

    cv2.line(im, p1, p2, (255,0,0), 2)
#     plt.imshow(im[...,::-1])
    cv2.imwrite('render/%07d.jpg'%ind,im)

NameError: name 'predictor' is not defined

In [23]:
video = 'E:/mesh2face/pose_estimation/optimize_v2/video.mp4'
video_images = 'E:/mesh2face/pose_estimation/optimize_v2/'

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video, fourcc, 5, (1280,720))
List = sorted(os.listdir(video_images))
 
for item in List:
    if not item.endswith('g'):
        continue
    img = cv2.imread(os.path.join(video_images,item))
    out.write(img)
out.release()